# Case Study: Movie Data Analysis

This notebook uses a dataset from the MovieLens website.

* Data SourceL MovieLens web site (filename: ml-20m.zip)
* Location https://grouplens.org/datasets/movielens


First Let's explore the folder to see what files we have in the dataset

In [1]:
!ls -la ./movielens

total 855092
drwxr-xr-x 1 aydin 197609         0 Mar  5 23:15 .
drwxr-xr-x 1 aydin 197609         0 Mar  8 00:07 ..
-rw-r--r-- 1 aydin 197609 323544381 Jan  3 23:41 genome-scores.csv
-rw-r--r-- 1 aydin 197609     18103 Jan  3 23:41 genome-tags.csv
-rw-r--r-- 1 aydin 197609    570090 Jan  3 23:41 links.csv
-rw-r--r-- 1 aydin 197609   1397542 Jan  3 23:41 movies.csv
-rw-r--r-- 1 aydin 197609 533444411 Jan  3 23:41 ratings.csv
-rw-r--r-- 1 aydin 197609     10261 Jan  3 23:41 README.txt
-rw-r--r-- 1 aydin 197609  16603996 Jan  3 23:41 tags.csv


# Use Pandas to read data

In this notebook, we will be using three CSV files:

* ratings.csv: userId, movieId, rating, timestamp
* tags.csv: userId, movieId, tag, timestamp
* movies.csv: movieId, title, genres

In [2]:
import pandas as pd

In [3]:
movie_data = pd.read_csv('./movielens/movies.csv', sep=',')

In [4]:
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
#Timestamps represent seconds since midnight Coordinated Universal Time (UTC)
tags = pd.read_csv('./movielens/tags.csv', sep=",")
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [6]:
ratings = pd.read_csv('./movielens/ratings.csv', sep=",")
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


### We do not need timestamp column right now but we will get back to it later on.

In [7]:
del ratings['timestamp']
del tags['timestamp']

# Descriptive Statistics

In [8]:
ratings['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [9]:
ratings['rating'].mean()

3.5255285642993797

In [10]:
ratings['rating'].min(), ratings['rating'].max(), ratings['rating'].std()

(0.5, 5.0, 1.0519889192942424)

In [11]:
ratings['rating'].mode()

0    4.0
dtype: float64

## Quick sanity check

Here we are verifying adequacy of ratings by checking if there are any ratings greater than 5 or less than 0. It is unneccessary since we know min and max ratings values but this is an alternative way.

In [12]:
filter_1 = ratings['rating'] > 5

In [13]:
filter_1.any()

False

In [14]:
filter_2 = ratings['rating'] > 0

In [15]:
filter_2.all()

True

# Data Cleaning: Handling missing data

In [16]:
movie_data.shape

(27278, 3)

In [17]:
movie_data.isnull().any()

movieId    False
title      False
genres     False
dtype: bool

In [18]:
ratings.shape

(20000263, 3)

In [19]:
ratings.isnull().any()

userId     False
movieId    False
rating     False
dtype: bool

In [20]:
tags.shape

(465564, 3)

In [21]:
tags.isnull().any()

userId     False
movieId    False
tag         True
dtype: bool

In [22]:
null_ = tags['tag'].isnull()
null_

0         False
1         False
2         False
3         False
4         False
          ...  
465559    False
465560    False
465561    False
465562    False
465563    False
Name: tag, Length: 465564, dtype: bool

In [23]:
null_tags = tags[null_]
null_tags

,userId,movieId,tag
373276,116460,123,NaN
373277,116460,346,NaN
373281,116460,1184,NaN
373288,116460,1785,NaN
373289,116460,2194,NaN
373291,116460,2691,NaN
373299,116460,4103,NaN
373301,116460,4473,NaN
373303,116460,4616,NaN
373319,116460,7624,NaN


In [24]:
null_tag_movies = null_tags['movieId']
pd.DataFrame(movie_data, index=null_tag_movies)

,movieId,title,genres
movieId,,,
123,125.0,Flirting With Disaster (1996),Comedy
346,350.0,"Client, The (1994)",Drama|Mystery|Thriller
1184,1210.0,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
1785,1869.0,Black Dog (1998),Action|Thriller
2194,2279.0,Urban Legend (1998),Horror|Thriller
2691,2777.0,Cobra (1925),Drama
4103,4197.0,Real Life (1979),Comedy
4473,4568.0,Best of the Best (1989),Action
4616,4711.0,Theremin: An Electronic Odyssey (1993),Documentary


### Since we have null values for tags, let's drop them to have a cleaner dataset

In [25]:
tags = tags.dropna()

In [26]:
tags.isnull().any()

userId     False
movieId    False
tag        False
dtype: bool

In [27]:
tags.shape

(465548, 3)

### Extract year from title e.g (1995)

In [28]:
movie_data['year'] = movie_data['title'].str.extract(".*\((.*)\).*", expand=True)

In [29]:
movie_data.tail()

,movieId,title,genres,year
27273,131254,Kein Bund für's Leben (2007),Comedy,2007
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,2002
27275,131258,The Pirates (2014),Adventure,2014
27276,131260,Rentun Ruusu (2001),(no genres listed),2001
27277,131262,Innocence (2014),Adventure|Fantasy|Horror,2014


# Merge Data - average movie ratings over time


In [30]:
average_rating = ratings[['movieId', 'rating']].groupby('movieId', as_index=False).mean()
average_rating.tail()

,movieId,rating
26739,131254,4.0
26740,131256,4.0
26741,131258,2.5
26742,131260,3.0
26743,131262,4.0


In [31]:
joined = movie_data.merge(average_rating, on='movieId', how='inner')
joined.head()

,movieId,title,genres,year,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.861393
4,5,Father of the Bride Part II (1995),Comedy,1995,3.064592


In [32]:
joined.head()

,movieId,title,genres,year,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.861393
4,5,Father of the Bride Part II (1995),Comedy,1995,3.064592


## Specify Data to be used in Twitter API request

In [33]:
rating_mask = joined.rating  > 4
year_mask = joined.year > '2010'

In [34]:
final_data = joined[rating_mask & year_mask]
final_data = final_data.reset_index().drop(columns='index')

In [35]:
import numpy as np
titles = np.array(final_data.title)
for index, title in np.ndenumerate(titles):
    final_data.loc[index, 'title'] = title.split("(")[0]
final_data


,movieId,title,genres,year,rating
0,91007,I Want to Be a Soldier,Drama,2011,4.500000
1,91529,"Dark Knight Rises, The",Action|Adventure|Crime|IMAX,2012,4.000210
2,92259,Intouchables,Comedy|Drama,2011,4.132396
3,92535,Louis C.K.: Live at the Beacon Theater,Comedy,2011,4.062349
4,94466,Black Mirror,Drama|Sci-Fi,2011,4.182990
...,...,...,...,...,...
133,130219,The Dark Knight,Action|Crime|Drama|Thriller,2011,4.500000
134,130294,Drevo,Drama,2014,4.500000
135,130374,India's Daughter,Documentary,2015,4.500000
136,130524,Flying Home,Romance,2014,4.500000


In [36]:
import numpy as np
final_data = final_data.drop(final_data[final_data.title.str.contains(", The")].index, inplace = False)
final_data = final_data.reset_index().drop(columns='index')
titles = np.array(final_data.title)
for index, title in np.ndenumerate(titles):
    final_data.loc[index, 'hashtags'] = "#" +  title.replace(" ", "")

In [37]:
final_data

,movieId,title,genres,year,rating,hashtags
0,91007,I Want to Be a Soldier,Drama,2011,4.500000,#IWanttoBeaSoldier
1,92259,Intouchables,Comedy|Drama,2011,4.132396,#Intouchables
2,92535,Louis C.K.: Live at the Beacon Theater,Comedy,2011,4.062349,#LouisC.K.:LiveattheBeaconTheater
3,94466,Black Mirror,Drama|Sci-Fi,2011,4.182990,#BlackMirror
4,96606,Samsara,Documentary,2011,4.006944,#Samsara
...,...,...,...,...,...,...
117,130219,The Dark Knight,Action|Crime|Drama|Thriller,2011,4.500000,#TheDarkKnight
118,130294,Drevo,Drama,2014,4.500000,#Drevo
119,130374,India's Daughter,Documentary,2015,4.500000,#India'sDaughter
120,130524,Flying Home,Romance,2014,4.500000,#FlyingHome


# Twitter API access

In [40]:
# Dependencies

import os
import tweepy as tw
import json
from pprint import pprint
import config

In [41]:
auth = tw.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [42]:
# if not os.path.exists('secret_twitter_credentials.pkl'):
#     Twitter={}
#     Twitter['Consumer Key'] = config.consumer_key
#     Twitter['Consumer Secret'] = config.consumer_secret
#     Twitter['Access Token'] = config.access_token
#     Twitter['Access Token Secret'] = config.access_token_secret
#     with open('secret_twitter_credentials.pkl','wb') as f:
#         pickle.dump(Twitter, f)
# else:
#     Twitter=pickle.load(open('secret_twitter_credentials.pkl','rb'))

In [43]:
# import twitter

# auth = twitter.oauth.OAuth(Twitter['Access Token'],
#                            Twitter['Access Token Secret'],
#                            Twitter['Consumer Key'],
#                            Twitter['Consumer Secret'])

# twitter_api = twitter.Twitter(auth=auth)

### Where on Earth ID number.. this helps to find location based on ID

This was not needed since we found different way to pull data from twitter but still nice to have for future references


In [44]:
# WORLD_WOE_ID = 1
# US_WOE_ID = 23424977
# LOCAL_WOE_ID=2357024 # Atlanta WOEID

In [46]:
# world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
# us_trends = twitter_api.trends.place(_id=US_WOE_ID)
# local_trends = twitter_api.trends.place(_id=LOCAL_WOE_ID)

In [47]:
def twitter_pull(hash):
    users_and_text = {}
    tweet_list = ''
    tweets = tw.Cursor(api.search,
        q=hash,
            lang="en").items(1)
    for tweet in tweets:
        users_and_text["hashtag"] = hash
        users_and_text["user_name"] = tweet.user.screen_name
        users_and_text["tweet"] = tweet.text
        users_and_text["location"] = tweet.user.location
#     users_and_text = [[tweet.user.screen_name, tweet.text, tweet.user.location] for tweet in tweets]
    return users_and_text
    

In [48]:
twits = []
for hash in final_data.hashtags:
    twits.append(twitter_pull(hash))

In [49]:
for tweet in twits:
    print(tweet)

{}
{'hashtag': '#Intouchables', 'user_name': 'regos_istvan', 'tweet': 'Thinking about ‘Parasite’ I couldn’t help but wonder why ‘Intouchables’ or ‘Amélie’ didn’t win Best Picture, Best A… https://t.co/bjB3W33Ere', 'location': 'Budapest, Hungary'}
{}
{'hashtag': '#BlackMirror', 'user_name': 'junheefairy', 'tweet': "I've just watched episode S01E02 of Black Mirror! #BlackMirror #tvtime https://t.co/EL7KMUlkX6 https://t.co/z9guDVsqsU", 'location': 'he/him '}
{'hashtag': '#Samsara', 'user_name': 'Turyamusic', 'tweet': 'Thank you @smh for this kind review of #Samsara: https://t.co/z2P3PVxJgC', 'location': 'London'}
{}
{'hashtag': '#TwoRabbits', 'user_name': 'cbnetwork19', 'tweet': 'Glendale Running 10.31 miles training run this morning... https://t.co/ogtkODnhK9\n#teamrce #tenmiles #tworabbits', 'location': 'Arizona, USA'}
{'hashtag': '#Guilty', 'user_name': 'rameshlaus', 'tweet': 'RT @LetsOTT: #Guilty Review: Has A Promise Which Is Not Fully Realised \n\nLetsOTT Rating: 2/5\n\nhttps://t.co

In [ ]:
hashtags